In [23]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import scipy
import seaborn as sns
import pandas as pd


# custom imports
import importlib
import ff_utils as utils
importlib.reload(utils)

# set params
league_id = '83174673'
swid = '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}'
espn_s2 = 'AEAy3mkAv%2FsnK2YbgEVRFKpOw6%2ByQVeXH5BDBOlzgAiBg646ZtRuneInbY1z9QYrt1Ws8r6Cc8pS0g%2BVi7HmU0b%2F0lxgkP9csOvk9S01pusHVMJOZ9GfMZblZ01U9NjD46R2N%2BI21guim1Lga4TfVKqE6rCLMJvQszv5ZKdFgjngR98gxiClp9R3ZxzJ7cTLJufuir4JGnzKZ9RwOCL1%2BOaFy8qFEGwEVNGkuMY4Hj4zSR%2BL9a5fatMYEWNXvk4muhbgckHixSZiqvSnL6pyniu98SRFEpigMIkWgSJ%2FTfNvCg%3D%3D'
year = 2021

# load data
FF = utils.FF(league_id=league_id, swid=swid, espn_s2=espn_s2, year=year)

In [24]:
# save local copy of data
# FF.pull_and_save_data()
FF.load_data()

,Unnamed: 0,Week,Team,Player,Slot,Pos,Status,Proj,Actual
0,0,1,1,Christian McCaffrey,2,RB,INJURY_RESERVE,21.299062,27.7
1,1,1,1,DK Metcalf,4,WR,ACTIVE,16.368928,16.0
2,2,1,1,Darren Waller,6,TE,ACTIVE,14.161294,26.5
3,3,1,1,D'Andre Swift,23,Flex,ACTIVE,13.547044,24.4
4,4,1,1,Chris Godwin,4,WR,INJURY_RESERVE,13.843645,23.5
...,...,...,...,...,...,...,...,...,...
2237,2237,14,10,Mike Gesicki,20,Bench,ACTIVE,0.000000,NaN
2238,2238,14,10,Alexander Mattison,2,RB,ACTIVE,6.305252,2.7
2239,2239,14,10,Mike Davis,20,Bench,QUESTIONABLE,7.533525,13.6
2240,2240,14,10,Russell Gage,4,WR,ACTIVE,10.590785,10.4


In [25]:
# some overall decoration settings
sns.set_style('darkgrid')
plt.xkcd()
weeks = list(range(1,15))
linewidth=3

In [37]:
# plot sum player score by week for each position. pos=None means whole team
pos = 'TE'
plt.figure(figsize=(12, 8))


# constants
num_teams = len(FF.get_team_id(None).keys())
num_weeks = len(weeks)
num_pos_slots = FF.get_number_of_position_slots(pos)

# initialize output data structure
data_array = np.empty(shape=(num_teams, num_weeks, num_pos_slots))

# iterate teams and weeks
for t in range(num_teams):
    for w in range(num_weeks):

        # iterate players in position, +1 for 1-indexing when looking up team/week score
        tid = t + 1
        wid = w + 1
        players = FF.get_team_starters(tid, wid, pos=pos)
        for p in range(len(players)):
            rank = FF.get_player_score(players[p], wid)
            data_array[t, w, p] = rank

# get average
avg = np.nansum(np.nanmean(data_array, axis=0), axis=1)

# plot each team
for t in range(num_teams):

    to_plot = np.nansum(data_array[t,:,:], axis=1)

    # get number of weeks above average
    avg_relative_to_avg = np.round(np.nanmean(to_plot - avg), decimals=1)
    tid = t+1
    color = FF.get_team_color(tid)
    nn = FF.get_team_nickname(tid)
    num_p = FF.get_num_players_started(tid, pos)
    plt.plot(weeks, to_plot, color=color, linewidth=linewidth, label=nn + '(' + str(num_p) + 'Δ{})'.format(avg_relative_to_avg))


# plot average
plt.plot(weeks, avg, color='k', linewidth=linewidth, label='average')

# decorate
#plt.legend(ncol=4, loc='lower center')
plt.legend(ncol=4, loc='upper center', bbox_to_anchor=(0.5, -0.1))
plt.xticks(weeks)
plt.ylabel('Total scoring by week')
plt.xlabel('Week')
plt.title('Scoring by team')
plt.title('Scoring of {} by team'.format(pos))
#plt.ylim([60, 240])
#plt.ylim([0, 50])


KeyError: 0

<Figure size 864x576 with 0 Axes>